![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [5]:
import sqlite3
import pandas as pd

#### Load a DB object with `connect` and `cursor`

In [6]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [7]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [8]:
cursor.execute("SELECT * FROM airports")

#### Use description

In [9]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [10]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [11]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall() 
print(*info, sep= "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [12]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall() 
print(*info, sep= "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

In [ ]:
cursor.execute("""SELECT name FROM airlines WHERE country = 'United Kingdom' AND active = 'Y'""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

In [ ]:
cursor.execute("""SELECT DISTINCT country FROM airports""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [18]:
cursor.execute("""SELECT routes.*, airports.latitude, airports.longitude FROM routes LEFT JOIN airports ON routes.index = airports.index""")
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]
df

OperationalError: near "index": syntax error

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airports?
- Which countries have the highest amount of inactive airports?
- What about airports by timezones?

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [14]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [15]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
345,A443876,Max,05/06/2019 04:56:00 PM,05/06/2019 04:56:00 PM,03/21/2006,Adoption,NaN,Cat,Neutered Male,13 years,Domestic Shorthair Mix,Blue Tabby/White
669,A793215,Max,04/30/2019 11:49:00 AM,04/30/2019 11:49:00 AM,04/21/2017,Return to Owner,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Black/White
819,A731802,Max,04/26/2019 06:17:00 PM,04/26/2019 06:17:00 PM,07/27/2015,Adoption,NaN,Dog,Neutered Male,3 years,Maltese Mix,White
820,A789320,Max,04/26/2019 06:01:00 PM,04/26/2019 06:01:00 PM,08/19/2017,Adoption,NaN,Dog,Neutered Male,1 year,Pit Bull Mix,White/Black
1158,A793073,Max,04/19/2019 01:55:00 PM,04/19/2019 01:55:00 PM,04/19/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Boxer Mix,Brown/Tricolor


In [16]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794145,NaN,05/13/2019 04:36:00 PM,05/13/2019 04:36:00 PM,03/04/2019,Transfer,Partner,Dog,Intact Male,2 months,Shih Tzu Mix,White/Brown
1,A794788,Rocco,05/13/2019 11:59:00 AM,05/13/2019 11:59:00 AM,08/13/2018,Return to Owner,NaN,Dog,Neutered Male,8 months,Miniature Pinscher,Black/Tan
2,A794472,NaN,05/13/2019 11:45:00 AM,05/13/2019 11:45:00 AM,05/08/2017,Transfer,Partner,Dog,Spayed Female,2 years,Bichon Frise Mix,White
3,A794043,Pantera,05/13/2019 11:27:00 AM,05/13/2019 11:27:00 AM,01/03/2019,Adoption,NaN,Dog,Spayed Female,4 months,Pointer/Labrador Retriever,Black/White
4,A793688,NaN,05/13/2019 10:28:00 AM,05/13/2019 10:28:00 AM,11/28/2012,Disposal,NaN,Dog,Intact Male,6 years,Bloodhound,Brown


## Goal 6: Transfering from sqlite to pandas

In [17]:
conn = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", conn)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

How many tables are in the database? 11
What's the primary key of each table? 
* AlbumID
* ArtistID
* CustomerID
* EmployeeID
* GenreID
* InvoiceID
* InvoiceLineID
* MediaTypeID
* PlaylistId
* PlaylistId/TrackID
* TrackID
What foreign keys join the tables together?

[(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[(0, 0, 'Employee', 'SupportRepId', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Employee', 'ReportsTo', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[(0, 0, 'Customer', 'CustomerId', 'CustomerId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Invoice', 'InvoiceId', 'InvoiceId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[]
[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Playlist', 'PlaylistId', 'PlaylistId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE'), (2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')]

If you had to draw a schema of how the tables are connected, what would it look like?



In [81]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cursor = conn.cursor()
df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
df
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cursor.fetchall())

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [75]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE key='primary';").fetchall()


OperationalError: no such column: key

In [120]:
for i in tables:
    print(cursor.execute(f"PRAGMA foreign_key_list({i[0]})").fetchall())

[(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[(0, 0, 'Employee', 'SupportRepId', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Employee', 'ReportsTo', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[(0, 0, 'Customer', 'CustomerId', 'CustomerId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Invoice', 'InvoiceId', 'InvoiceId', 'NO ACTION', 'NO ACTION', 'NONE')]
[]
[]
[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Playlist', 'PlaylistId', 'PlaylistId', 'NO ACTION', 'NO ACTION', 'NONE')]
[(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE'), (2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [110]:
 cursor.execute("PRAGMA foreign_key_list('Album')").fetchall()

[(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')]

In [105]:
# def to_csv():
#    db = sqlite3.connect('database.db')
#    cursor = db.cursor()
#    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#    tables = cursor.fetchall()
#    for table_name in tables:
#        table_name = table_name[0]
#        table = pd.read_sql_query("SELECT * from %s" % table_name, db)
#        table.to_csv(table_name + '.csv', index_label='index')

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table_name in tables:
#     l = list(table_name.primary)
    print(table_name)

('Album',)
('Artist',)
('Customer',)
('Employee',)
('Genre',)
('Invoice',)
('InvoiceLine',)
('MediaType',)
('Playlist',)
('PlaylistTrack',)
('Track',)


In [66]:
cursor.execute("SHOW columns FROM tablename WHERE 'Key' = 'PRI'").fetchall()
# cursor.execute("SELECT * FROM Invoice")
# cursor.description

OperationalError: near "SHOW": syntax error

## Reflection

In [156]:
cursor.execute("SELECT * FROM Track").fetchall()
cursor.description

(('TrackId', None, None, None, None, None, None),
 ('Name', None, None, None, None, None, None),
 ('AlbumId', None, None, None, None, None, None),
 ('MediaTypeId', None, None, None, None, None, None),
 ('GenreId', None, None, None, None, None, None),
 ('Composer', None, None, None, None, None, None),
 ('Milliseconds', None, None, None, None, None, None),
 ('Bytes', None, None, None, None, None, None),
 ('UnitPrice', None, None, None, None, None, None))